In [8]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

# 定义linear elastic模型的应力计算函数
# Define the stress calculation function of the linear elastic model
def linear_elastic_stress(strain, E, nu):
    D = E/(1 - nu**2)*np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1 - nu)/2]])
    stress = np.dot(D, strain)
    return stress

# 生成样本数据（应变和应力）
# generate sample data (strain and stress)
n_samples = 1000
E_true = 1.0e6
nu_true = 0.3
noise_std = 100.0
strain_samples = np.random.uniform(-0.01, 0.01, size=(n_samples, 3))
stress_samples = np.zeros((n_samples, 3))
for i in range(n_samples):
    stress_samples[i] = linear_elastic_stress(strain_samples[i], E_true, nu_true)
    noise = np.random.normal(scale=noise_std, size=3)
    stress_samples[i] += noise

# 定义似然函数（概率模型）
# Define the likelihood function (probability model)
def likelihood(params, strain, stress):
    E, nu, noise_std = params
    n_samples = strain.shape[0]
    ll = 0
    for i in range(n_samples):
        model_stress = linear_elastic_stress(strain[i], E, nu)
        ll += norm.logpdf(stress[i], loc=model_stress, scale=noise_std).sum()
    return -ll

# 最大化似然函数，得到最优的模型参数估计
# Maximize the likelihood function to get the optimal model parameter estimation
# x0=[initial value]
res = minimize(likelihood, x0=[1.0e6, 0.3, 100.0], args=(strain_samples, stress_samples), method='L-BFGS-B')
E_est, nu_est, noise_std_est = res.x
print('Estimated E:', E_est)
print('Estimated nu:', nu_est)
print('Estimated noise_std:', noise_std_est)


Estimated E: 1000000.0000000002
Estimated nu: 0.2997353982488295
Estimated noise_std: 99.99999997279618


已知stress，求参数E，nu，noise